In [52]:
import os
import pandas as pd
import numpy as np
import json

In [53]:
# Get the current working directory
CURRENT_DIRECTORY = os.getcwd()
# Get the parent directory
PARENT_DIRECTORY = os.path.dirname(CURRENT_DIRECTORY)
PARENT_DIRECTORY = os.path.dirname(PARENT_DIRECTORY)

# Open the config file and load its content into a dictionary
config_file = open(PARENT_DIRECTORY + '\\config\\config.json')
CONFIG_DATA = json.load(config_file)

# Close the file after loading the data
config_file.close()

In [54]:
WINDOW_SIZE = CONFIG_DATA['WINDOW_SIZE']
NUMBER_OF_FEATURES = 4

CLASSIFICATIONS = CONFIG_DATA['CLASSES_MOTIONTYPE']

LABEL_COLUMN = 'Class_MotionType'

# processed data folder path
DATA_FOLDER = PARENT_DIRECTORY + '\\processed-training-data\\4-PROCESSED-DATA\TRAIN2\\'

# save numpy data folder
NUMPY_DATA_FOLDER_FILE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\'

# v0 data (not normalized, use to generate more data)
ALL_X_TRAIN_CLASSIFY_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-X-TRAIN-CLASSIFY-MOTIONTYPE-STEPS-V0.npy'
ALL_Y_TRAIN_CLASSIFY_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-Y-TRAIN-CLASSIFY-MOTIONTYPE-STEPS-V0.npy'

In [55]:
LABEL_TO_CATEGORY = {label: category for category, label in enumerate(CLASSIFICATIONS)}

print(LABEL_TO_CATEGORY)

{'LAR': 0, 'SML': 1}


In [56]:
file_names = [file for file in os.listdir(DATA_FOLDER) if file.endswith('.xlsx') and "SIDESTEPS" not in file and "STEPS" in file and os.path.isfile(os.path.join(DATA_FOLDER, file))]

print(file_names)

['PROC-TRAIN2-STEPS-LR-LAR-100BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-110BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-112BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-120BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-124BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-164BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-196BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-200BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-220BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-224BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-30BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-31BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-40BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-41BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-49BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-50BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-55BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-56BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-60BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-62BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-80BPM.xlsx', 'P

In [57]:
def normalizeSensorData(sensor_input):
    return (sensor_input + 180)/360

In [58]:
def count_sequences_above_threshold(data, threshold):
    num_sensors = data.shape[1]
    counts = np.zeros(num_sensors, dtype=int)

    for sensor_index in range(num_sensors):
        count = 0
        sequence_above_threshold = False

        for value in data[:, sensor_index]:
            if value > threshold:
                if not sequence_above_threshold:
                    count += 1
                    sequence_above_threshold = True
            else:
                sequence_above_threshold = False

        counts[sensor_index] = count

    return counts

In [59]:
def df2Xy(df, windowSize=5):
  X = []
  y = []

  # NORMALIZE THE SENSOR DATA
  #df["L_Pitch_Delta"] = df["L_Pitch_Delta"].apply(normalizeSensorData)
  #df["L_Roll_Delta"] = df["L_Roll_Delta"].apply(normalizeSensorData)
  #df["R_Pitch_Delta"] = df["R_Pitch_Delta"].apply(normalizeSensorData)
  #df["R_Roll_Delta"] = df["R_Roll_Delta"].apply(normalizeSensorData)

  for i in range( len(df) - windowSize + 1):
    # inputs: X rows
    # form a new input which has size of our windowSize
    input_data_list_readings = []

    # loop through each row in our windowsize
    for j in range(windowSize):
        # fetch sensor data for this row
        row_values_readings = df.loc[i + j, ['L_Pitch', 'L_Roll', 'R_Pitch', 'R_Roll']].values.tolist()

        # add row values to the input
        input_data_list_readings.append(row_values_readings)

    # turnn list into array to do arthimetic
    raw_sensor_data = np.array(input_data_list_readings)

    max_sensor_reading = np.max(raw_sensor_data, axis=0)

    input_data_array = np.copy(max_sensor_reading)

    if(np.isnan(input_data_array).any() == False):
      # turn back to list
      input_data_list = input_data_array.tolist()

      # add our input to our total inputs, marked as X
      X.append(input_data_list)

      # outputs: y labels
      label = df.loc[i + (windowSize - 1), [LABEL_COLUMN]].values.tolist()

      y.append(label)

  return (np.array(X), np.array(y))

In [60]:
ALL_X_TRAIN = np.empty((0, NUMBER_OF_FEATURES))  # List to store all X training data
ALL_Y_TRAIN = np.empty((0))  # List to store all Y training data

In [61]:
def processData(ALL_X, ALL_Y):
    for fileName in file_names:
        # Read the Excel file
        df = pd.read_excel(DATA_FOLDER + fileName)

        xTrain, yTrain = df2Xy(df, WINDOW_SIZE)

        yTrain = yTrain.reshape(-1)
        
        ALL_X = np.concatenate([ALL_X, xTrain])
        ALL_Y = np.concatenate([ALL_Y, yTrain])
        
        print('xTrain.shape:', xTrain.shape)
        print('ALL_X_TRAIN.shape:', ALL_X.shape, ' ALL_Y_TRAIN.shape:', ALL_Y.shape)
        print('-------------------------------')

    # Create the folder if it doesn't exist
    if not os.path.exists(NUMPY_DATA_FOLDER_FILE_PATH):
        os.makedirs(NUMPY_DATA_FOLDER_FILE_PATH)

    # convert to numerical labels (originally text labels)
    numerical_label = np.vectorize(LABEL_TO_CATEGORY.get)(ALL_Y)

    # Assuming your numpy array is called 'data_array'
    np.save(ALL_X_TRAIN_CLASSIFY_PATH, ALL_X)
    np.save(ALL_Y_TRAIN_CLASSIFY_PATH, numerical_label)

In [62]:
processData(ALL_X_TRAIN, ALL_Y_TRAIN)

ALL_X_TRAIN_CLASSIFY = np.load(ALL_X_TRAIN_CLASSIFY_PATH)
ALL_Y_TRAIN_CLASSIFY = np.load(ALL_Y_TRAIN_CLASSIFY_PATH, allow_pickle=True)

print(ALL_X_TRAIN_CLASSIFY.shape)
print(ALL_Y_TRAIN_CLASSIFY.shape)

xTrain.shape: (3381, 4)
ALL_X_TRAIN.shape: (3381, 4)  ALL_Y_TRAIN.shape: (3381,)
-------------------------------
xTrain.shape: (3274, 4)
ALL_X_TRAIN.shape: (6655, 4)  ALL_Y_TRAIN.shape: (6655,)
-------------------------------
xTrain.shape: (4560, 4)
ALL_X_TRAIN.shape: (11215, 4)  ALL_Y_TRAIN.shape: (11215,)
-------------------------------
xTrain.shape: (3773, 4)
ALL_X_TRAIN.shape: (14988, 4)  ALL_Y_TRAIN.shape: (14988,)
-------------------------------
xTrain.shape: (3181, 4)
ALL_X_TRAIN.shape: (18169, 4)  ALL_Y_TRAIN.shape: (18169,)
-------------------------------
xTrain.shape: (3072, 4)
ALL_X_TRAIN.shape: (21241, 4)  ALL_Y_TRAIN.shape: (21241,)
-------------------------------
xTrain.shape: (4505, 4)
ALL_X_TRAIN.shape: (25746, 4)  ALL_Y_TRAIN.shape: (25746,)
-------------------------------
xTrain.shape: (3796, 4)
ALL_X_TRAIN.shape: (29542, 4)  ALL_Y_TRAIN.shape: (29542,)
-------------------------------
xTrain.shape: (3381, 4)
ALL_X_TRAIN.shape: (32923, 4)  ALL_Y_TRAIN.shape: (32923,)
-

In [63]:
print(np.max(ALL_X_TRAIN_CLASSIFY))

61.0
